<a href="https://colab.research.google.com/github/GustavoGatti/BreastCancerSiameseNeuralNetwork/blob/main/SNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==2.0.0-rc0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, Lambda, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from keras.regularizers import l2

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from numpy import mean
from numpy import std
from numpy import ndarray
import numpy.random as rng
import random
from keras.layers import add

In [ ]:
from keras import backend as K
import keras as KER

ImportError: ignored

In [ ]:
from keras.layers import merge

In [ ]:
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

In [ ]:
def cosine(vector):
  x, y = vector
  dotx = K.sum(x*x, axis=1,keepdims=True)
  doty = K.sum(y*y, axis=1,keepdims=True)
  dotxy = K.sum(x*y, axis=1,keepdims=True)
  cos = (dotxy/(K.sqrt(dotx) * K.sqrt(doty)))
  return cos

In [ ]:
def eucl_dist_output_shape(shapes):
  shape1, shape2 = shapes
  return shape1[0], 1

In [ ]:
def contrastive_loss(y_true,y_pred):
  return K.mean(y_true*(2*K.exp(-(1/K.square(y_pred-1.2)))) + (1 - y_true)*(2*K.exp(-(1/K.square(y_pred+1.2)))))

In [ ]:
def logistic_loss(y_true,y_pred):
  return K.mean(K.log(1 * K.exp(-1*y_pred*y_true)))

In [ ]:
def compute_accuracy(y_true,y_pred):
  classified = K.round(y_pred)
  return K.mean(K.equal(classified, y_true))

In [ ]:
def compute_final_accuracy(predictions, labels, print_prediction = False):
  if print_prediction:
    print(predictions)

  classified = np.round(predictions.ravel())
  return np.mean(classified == labels)

In [ ]:
def difference(vector):
  x, y = vector
  return x-y

In [ ]:
def cos_distance(vector):
  return 1-cosine(vector)

In [ ]:
def distance(cos):
  return 1 - K.abs(cos)

In [ ]:
def create_base_network(input_shape):
  model = Sequential()
  model.add(keras.layers.Reshape(input_shape = (9, 1), target_shape=(9,1)))
  model.add(Conv1D(filters=32, kernel_size=2, activation = 'relu', input_shape = (9,1)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Conv1D(filters=64, kernel_size=2, activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(64, activation = 'relu'))
  model.add(Dropout(0.5))

  model.add(Dense(1, activation = 'sigmoid'))
  
  return model

In [ ]:
def Transformacao(dataset):

  novo_data = pd.DataFrame(data=dataset)
  novo_data.drop('Classification',inplace=True,axis=1)  
  data_x = novo_data 
  data_y = dataset.Classification
  x_train = np.asarray(data_x)
  y_train = np.asarray(data_y)
  print(x_train)
  test_x = []
  test_y = []
  for i in range(116):
    if(i<5):
      test_x.append(x_train[i])
      test_y.append(y_train[i])
    elif(i>112):
      test_x.append(x_train[i])
      test_y.append(y_train[i])


 # test_x = tf.cast(test_x, tf.float32)
  test_x = np.asarray(test_x)
  test_y = np.asarray(test_y)
  print(test_x.shape)
  print(test_y.shape)

  scaler = StandardScaler()
  test_x = scaler.fit_transform(test_x)
  x_train = scaler.fit_transform(x_train)
  x_train = x_train.reshape(116,9,1)
  test_x = test_x.reshape(8,9,1)

  return test_x, test_x, test_y, test_y, scaler

In [ ]:
def Montando_data_set(x_train,x_train1,y_train,y_train1):
  target = []
  trainx = []
  trainx1 =[]
  print(y_train)
  print(y_train1)
  print(len(x_train))
  print(len(x_train1))
  for i in range(8):
    for j in range(8):
      trainx.append(x_train[i])
      trainx1.append(x_train1[j])
      if(y_train[i] == y_train1[j]):
        target.append(1)
      else:
        target.append(0)
  trainx=np.asarray(trainx)
  trainx = tf.cast(trainx, tf.float32)
  trainx1=np.asarray(trainx1)
  trainx1 = tf.cast(trainx1, tf.float32)
  xtreino = [trainx,trainx1]
  target = np.asarray(target)
  target = tf.cast(target, tf.float32)

  return xtreino,target

In [ ]:
def Transformacao_teste(dataset):

  novo_data = pd.DataFrame(data=dataset)
  novo_data.drop('Classification',inplace=True,axis=1)  
  data_x = novo_data 
  data_y = dataset.Classification
  x_train = np.asarray(data_x)
  y_train = np.asarray(data_y)
  print(x_train)
  test_x = []
  test_y = []
  y = 0
  for i in range(116):
    if((i>7) & (i<15)):
      test_x.append(x_train[i])
      test_y.append(y_train[i])
      y = y+1
    elif((i>85) & (i<95) ):
      test_x.append(x_train[i])
      test_y.append(y_train[i])
      y = y+1

  print(y)
  test_x = tf.cast(test_x, tf.float32)
  #test_x = np.asarray(test_x)
  test_y = np.asarray(test_y)

  scaler = StandardScaler()
  test_x = scaler.fit_transform(test_x)
  print(test_x.shape)
  test_x = test_x.reshape(16,9,1)

  return test_x, test_x, test_y, test_y

In [ ]:
def Montando_data_set_test(x_train,x_train1,y_train,y_train1):
  target = []
  trainx = []
  trainx1 =[]
  print(y_train)
  print(y_train1)
  print(len(x_train))
  print(len(x_train1))
  for i in range(8):
    for j in range(16):
      trainx.append(x_train[i])
      trainx1.append(x_train1[j])
      if(y_train[i] == y_train1[j]):
        target.append(1)
      else:
        target.append(0)
  trainx=np.asarray(trainx)
  trainx = tf.cast(trainx, tf.float32)
  trainx1=np.asarray(trainx1)
  trainx1 = tf.cast(trainx1, tf.float32)
  xtreino = [trainx,trainx1]
  target = np.asarray(target)
  target = tf.cast(target, tf.float32)

  return xtreino,target

In [ ]:
initialize_bias = KER.initializers.RandomNormal(mean=0.0, stddev=0.1, seed=1221)

In [ ]:
def execute(epochs=1000):
  np.random.seed(100)
  random.seed(100)
  
  #dataset

  cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
  x, x1, y, y1, scaler = Transformacao(cancer)
  x_train, y_train = Montando_data_set(x,x1,y,y1)

  cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
  x12, x123, y12, y123 = Transformacao_teste(cancer)
  x_test, y_test = Montando_data_set_test(x,x123,y,y123)  


  input = (9,1)
  base_network_w = create_base_network(input)
  left_input = Input(shape=(input))
  right_input = Input(shape=(input)) 

  left = base_network_w(left_input)
  right = base_network_w(right_input)

  #both = merge([left, right], mode = cosine, output_shape = eucl_dist_output_shape)
  both = tf.keras.layers.Dot(axes=1)([left, right])

  prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(both)

  model = Model(inputs=[left_input,right_input],outputs=prediction)

 # model = Model(inputs=[left_input,right_input],outputs=both)

  loss = contrastive_loss
  optimizer = Adam(0.0005)

  #model.compile(loss=loss, optimizer = optimizer, metrics = [contrastive_loss, compute_accuracy])

  model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])

  hist = model.fit(x_train, y_train, epochs=epochs,validation_data=(x_test,y_test), verbose=1)

  return hist,model, x_test, y_test


In [ ]:
hist, model,x_test, y_test = execute()

In [ ]:
def plot_learningCurve(history, epochs):

  #Plot Training e validation accuracy values
  epoch_range = range(1, epochs+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

  #Plot training e validation accuracy loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningCurve(hist, 2000)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)